In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
clean_data = '/content/drive/MyDrive/Mini_project/cleaned_dataset.csv'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv(clean_data)

In [ ]:
df.drop(df.index[1000:], inplace=True)

In [ ]:
df.head()

,Id,ProfileName,Score,Summary,Text,review_clean
0,1,delmartian,5,Good Quality Dog Food,I have bought several of the Vitality canned d...,buy several vitality can dog food product find...
1,2,dll pa,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,product arrive labeled jumbo salt peanuts...th...
2,3,"Natalia Corres ""Natalia Corres""",4,"""Delight"" says it all",This is a confection that has been around a fe...,confection around century light pillowy citrus...
3,4,Karl,2,Cough Medicine,If you are looking for the secret ingredient i...,look secret ingredient robitussin believe find...
4,5,"Michael D. Bigham ""M. Wassir""",5,Great taffy,Great taffy at a great price. There was a wid...,great taffy great price wide assortment yummy ...


In [ ]:
import multiprocessing

from gensim.models import Word2Vec

In [ ]:
! pip install unidecode

In [ ]:
import pandas as pd
import numpy as np
import re
from re import sub
import multiprocessing
from unidecode import unidecode

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors


from time import time 
from collections import defaultdict

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [ ]:
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

In [ ]:
sent = [row for row in df.review_clean]
phrases = Phrases(sent, min_count=1, progress_per=50000)
bigram = Phraser(phrases)
sentences = bigram[sent]
sentences[1]

INFO - 17:04:23: collecting all words and their counts
INFO - 17:04:23: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 17:04:24: collected 882 word types from a corpus of 233049 words (unigram + bigrams) and 1000 sentences
INFO - 17:04:24: using 882 counts as vocab in Phrases<0 vocab, min_count=1, threshold=10.0, max_vocab_size=40000000>
INFO - 17:04:24: source_vocab length 882
INFO - 17:04:24: Phraser built with 1 phrasegrams


'product arrive labeled jumbo salt peanuts...the peanut actually small size unsalted sure error vendor intend represent product jumbo'

In [ ]:
start = time()

w2v_model.build_vocab(sentences, progress_per=50000)

print('Time to build vocab: {} mins'.format(round((time() - start) / 60, 2)))

INFO - 17:04:35: collecting all words and their counts
WARNING - 17:04:35: Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.
INFO - 17:04:35: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 17:04:35: collected 48 word types from a corpus of 233049 raw words and 1000 sentences
INFO - 17:04:35: Loading a fresh vocabulary
INFO - 17:04:35: effective_min_count=3 retains 43 unique words (89% of original 48, drops 5)
INFO - 17:04:35: effective_min_count=3 leaves 233043 word corpus (99% of original 233049, drops 6)
INFO - 17:04:35: deleting the raw counts dictionary of 48 items
INFO - 17:04:35: sample=1e-05 downsamples 39 most-common words
INFO - 17:04:35: downsampling leaves estimated 3747 word corpus (1.6% of prior 233043)
INFO - 17:04:35: estimated required memory for 43 words and 300 dimensions: 124700 bytes
INFO - 17:04:35: resetting layer weights


Time to build vocab: 0.0 mins


In [ ]:
start = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - start) / 60, 2)))

w2v_model.init_sims(replace=True)

INFO - 17:04:54: training model with 1 workers on 43 vocabulary and 300 features, using sg=0 hs=0 sample=1e-05 negative=20 window=4
INFO - 17:04:54: worker thread finished; awaiting finish of 0 more threads
INFO - 17:04:54: EPOCH - 1 : training on 233049 raw words (3718 effective words) took 0.1s, 57044 effective words/s
INFO - 17:04:54: worker thread finished; awaiting finish of 0 more threads
INFO - 17:04:54: EPOCH - 2 : training on 233049 raw words (3717 effective words) took 0.0s, 76795 effective words/s
INFO - 17:04:54: worker thread finished; awaiting finish of 0 more threads
INFO - 17:04:54: EPOCH - 3 : training on 233049 raw words (3791 effective words) took 0.0s, 79793 effective words/s
INFO - 17:04:54: worker thread finished; awaiting finish of 0 more threads
INFO - 17:04:54: EPOCH - 4 : training on 233049 raw words (3729 effective words) took 0.0s, 77942 effective words/s
INFO - 17:04:54: worker thread finished; awaiting finish of 0 more threads
INFO - 17:04:54: EPOCH - 5 : 

Time to train the model: 0.03 mins


In [ ]:
w2v_model.save("word2vec.model")

INFO - 17:05:12: saving Word2Vec object under word2vec.model, separately None
INFO - 17:05:12: not storing attribute vectors_norm
INFO - 17:05:12: not storing attribute cum_table
INFO - 17:05:12: saved word2vec.model


In [ ]:
file_export = df.copy()
file_export['old_text'] = file_export.review_clean
file_export.old_text = file_export.old_text.str.join(' ')
file_export.review_clean = file_export.review_clean.apply(lambda x: ' '.join(bigram[x]))
file_export.Score = file_export.Score.astype('int8')

In [ ]:
file_export[['review_clean', 'Score']].to_csv('NEWcleaned_dataset.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.cluster import KMeans

In [ ]:
word_vectors = Word2Vec.load("/content/word2vec.model").wv

INFO - 17:11:10: loading Word2Vec object from /content/word2vec.model
INFO - 17:11:10: loading wv recursively from /content/word2vec.model.wv.* with mmap=None
INFO - 17:11:10: setting ignored attribute vectors_norm to None
INFO - 17:11:10: loading vocabulary recursively from /content/word2vec.model.vocabulary.* with mmap=None
INFO - 17:11:10: loading trainables recursively from /content/word2vec.model.trainables.* with mmap=None
INFO - 17:11:10: setting ignored attribute cum_table to None
INFO - 17:11:10: loaded /content/word2vec.model


In [ ]:
model = KMeans(n_clusters=2, max_iter=1000, random_state=True, n_init=50).fit(X=word_vectors.vectors.astype('double'))

In [ ]:
word_vectors.similar_by_vector(model.cluster_centers_[1], topn=10, restrict_vocab=None)

INFO - 17:11:49: precomputing L2-norms of word weight vectors


[(')', 0.9999358057975769),
 (':', 0.9999303817749023),
 (',', 0.9999246597290039),
 ('h', 0.9999146461486816),
 ('b', 0.9999127984046936),
 ('>', 0.9999083280563354),
 ('x', 0.9998899698257446),
 ('(', 0.9998859167098999),
 ('?', 0.9998849630355835),
 ('&', 0.999879777431488)]

In [ ]:
positive_cluster_index = 1
positive_cluster_center = model.cluster_centers_[positive_cluster_index]
negative_cluster_center = model.cluster_centers_[1-positive_cluster_index]

In [ ]:
words = pd.DataFrame(word_vectors.vocab.keys())
words.columns = ['words']
words['vectors'] = words.words.apply(lambda x: word_vectors[f'{x}'])
words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])

In [ ]:
words['cluster_value'] = [1 if i==positive_cluster_index else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
words['sentiment_coeff'] = words.closeness_score * words.cluster_value

INFO - 17:12:49: NumExpr defaulting to 2 threads.


In [ ]:

words.head(10)

,words,vectors,cluster,cluster_value,closeness_score,sentiment_coeff
0,b,"[0.006674287, 0.07171218, -0.03071274, -0.0002...",1,1,75.691398,75.691398
1,u,"[0.0051192185, 0.07114216, -0.031233596, -0.00...",1,1,57.512168,57.512168
2,y,"[0.0070375404, 0.0694881, -0.031878166, -0.001...",0,-1,80.184397,-80.184397
3,,"[0.003566899, 0.07001784, -0.03069721, -0.0006...",1,1,31.117832,31.117832
4,s,"[0.007937334, 0.0684391, -0.033087328, -0.0025...",0,-1,49.617093,-49.617093
5,e,"[0.005335425, 0.06891587, -0.03138582, -0.0008...",1,1,40.949991,40.949991
6,v,"[0.0073090238, 0.07023648, -0.03271517, -0.002...",0,-1,65.421151,-65.421151
7,r,"[0.007818953, 0.07156993, -0.031813942, -0.000...",1,1,56.382018,56.382018
8,a,"[0.0071994313, 0.069396086, -0.03263359, -0.00...",0,-1,63.168262,-63.168262
9,l,"[0.0069516073, 0.06933905, -0.03215307, -0.001...",0,-1,90.602353,-90.602353


In [ ]:
words[['words', 'sentiment_coeff']].to_csv('sentiment_dictionary.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from IPython.display import display

In [ ]:
final_file = pd.read_csv('/content/NEWcleaned_dataset.csv')

In [ ]:
final_file.head()

,review_clean,Score
0,b u y s e v e r a l v i t a l i t y c a ...,5
1,p r o d u c t a r r i v e l a b e l e d ...,1
2,c o n f e c t i o n a r o u n d c e n t u ...,4
3,l o o k s e c r e t i n g r e d i e n t ...,2
4,g r e a t t a f f y g r e a t p r i c e ...,5


In [ ]:
sentiment_map = pd.read_csv('/content/sentiment_dictionary.csv')
sentiment_dict = dict(zip(sentiment_map.words.values, sentiment_map.sentiment_coeff.values))

In [ ]:
sentiment_map.head()

,words,sentiment_coeff
0,b,75.691398
1,u,57.512168
2,y,-80.184397
3,,31.117832
4,s,-49.617093


In [ ]:
file_weighting = final_file.copy()

In [ ]:
tfidf = TfidfVectorizer(tokenizer=lambda y: y.split(), norm=None)
tfidf.fit(file_weighting.review_clean)
features = pd.Series(tfidf.get_feature_names())
transformed = tfidf.transform(file_weighting.review_clean)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
def create_tfidf_dictionary(x, transformed_file, features):
    '''
    create dictionary for each input sentence x, where each word has assigned its tfidf score
    
    inspired  by function from this wonderful article: 
    https://medium.com/analytics-vidhya/automated-keyword-extraction-from-articles-using-nlp-bfd864f41b34
    
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer

    '''
    vector_coo = transformed_file[x.name].tocoo()
    vector_coo.col = features.iloc[vector_coo.col].values
    dict_from_coo = dict(zip(vector_coo.col, vector_coo.data))
    return dict_from_coo

def replace_tfidf_words(x, transformed_file, features):
    '''
    replacing each word with it's calculated tfidf dictionary with scores of each word
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer
    '''
    dictionary = create_tfidf_dictionary(x, transformed_file, features)   
    return list(map(lambda y:dictionary[f'{y}'], x.review_clean.split()))

In [ ]:
%%time
replaced_tfidf_scores = file_weighting.apply(lambda x: replace_tfidf_words(x, transformed, features), axis=1)#this step takes around 3-4 minutes minutes to calculate

CPU times: user 418 ms, sys: 8.49 ms, total: 427 ms
Wall time: 422 ms


In [ ]:
def replace_sentiment_words(word, sentiment_dict):
    '''
    replacing each word with its associated sentiment score from sentiment dict
    '''
    try:
        out = sentiment_dict[word]
    except KeyError:
        out = 0
    return out

In [ ]:
replaced_closeness_scores = file_weighting.review_clean.apply(lambda x: list(map(lambda y: replace_sentiment_words(y, sentiment_dict), x.split())))

In [ ]:
replacement_df = pd.DataFrame(data=[replaced_closeness_scores, replaced_tfidf_scores, file_weighting.review_clean, file_weighting.Score]).T
replacement_df.columns = ['sentiment_coeff', 'tfidf_scores', 'sentence', 'sentiment']
replacement_df['sentiment_rate'] = replacement_df.apply(lambda x: np.array(x.loc['sentiment_coeff']) @ np.array(x.loc['tfidf_scores']), axis=1)
replacement_df['prediction'] = (replacement_df.sentiment_rate>0).astype('int8')
replacement_df['sentiment'] = [1 if i==1 else 0 for i in replacement_df.sentiment]

In [ ]:
predicted_classes = replacement_df.prediction
y_test = replacement_df.sentiment

conf_matrix = pd.DataFrame(confusion_matrix(replacement_df.sentiment, replacement_df.prediction))
print('Confusion Matrix')
display(conf_matrix)

test_scores = accuracy_score(y_test,predicted_classes), precision_score(y_test, predicted_classes), recall_score(y_test, predicted_classes), f1_score(y_test, predicted_classes)

print('\n \n Scores')
scores = pd.DataFrame(data=[test_scores])
scores.columns = ['accuracy', 'precision', 'recall', 'f1']
scores = scores.T
scores.columns = ['scores']
display(scores)

Confusion Matrix


,0,1
0,840,62
1,93,5



 
 Scores


,scores
accuracy,0.845000
precision,0.074627
recall,0.051020
f1,0.060606
